In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [ ]:
data = pd.read_csv('/content/drive/My Drive/archive/IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [ ]:
english_stops = set(stopwords.words('english'))


In [ ]:
def load_dataset():
    df = pd.read_csv('/content/drive/My Drive/archive/IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
29729    [contains, spoilersthis, peter, watkins, film,...
8640     [first, i, played, second, monkey, island, gam...
6569     [i, heard, first, oceans, movie, sequels, begi...
2209     [this, good, plot, concept, poor, film, the, a...
34856    [national, lampoon, goes, movies, absolutely, ...
                               ...                        
49814    [the, gates, hell, opened, spit, film, closed,...
13693    [this, movie, poorly, written, hard, follow, f...
10085    [i, could, even, bring, watch, movie, end, i, ...
44681    [growing, joe, strummer, hero, mine, even, i, ...
9661     [quit, ahead, phrase, never, learn, typical, h...
Name: review, Length: 40000, dtype: object 

35662    [this, i, call, growth, movie, every, characte...
11974    [real, hoot, unintentionally, sidney, portier,...
13334    [the, acting, pretty, cheesy, people, area, de...
7580     [this, movie, seemed, like, going, better, end...
45121    [this, movie, nothing, religious, tract, promo...
 

In [ ]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))


In [ ]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[ 1274 30177   716 ...    60    46 19425]
 [   23     1   161 ...     0     0     0]
 [    1   462    23 ...     0     0     0]
 ...
 [    1    27    11 ...     0     0     0]
 [ 1795   841 24090 ...  4878  3502 24090]
 [ 4978  1352  4619 ...    66    24 12016]] 

Encoded X Test
 [[   8    1  574 ...    0    0    0]
 [  64 5380 3432 ...    0    0    0]
 [   2   43   88 ...    0    0    0]
 ...
 [ 228 1667 1054 ...    0    0    0]
 [   7    9    3 ...    0    0    0]
 [2015   51 4123 ...    0    0    0]] 

Maximum review length:  130


In [ ]:
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 130, 32)           2954528   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 2,979,425
Trainable params: 2,979,425
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
checkpoint = ModelCheckpoint(
    '/content/drive/My Drive/archive/lstm/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [ ]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])


Epoch 1/5
313/313 [==============================] - ETA: 0s - loss: 0.4748 - accuracy: 0.7481
Epoch 00001: accuracy improved from -inf to 0.74810, saving model to /content/drive/My Drive/archive/lstm/LSTM.h5
313/313 [==============================] - 18s 57ms/step - loss: 0.4748 - accuracy: 0.7481
Epoch 2/5
313/313 [==============================] - ETA: 0s - loss: 0.2129 - accuracy: 0.9240
Epoch 00002: accuracy improved from 0.74810 to 0.92395, saving model to /content/drive/My Drive/archive/lstm/LSTM.h5
313/313 [==============================] - 18s 57ms/step - loss: 0.2129 - accuracy: 0.9240
Epoch 3/5
313/313 [==============================] - ETA: 0s - loss: 0.1301 - accuracy: 0.9591
Epoch 00003: accuracy improved from 0.92395 to 0.95907, saving model to /content/drive/My Drive/archive/lstm/LSTM.h5
313/313 [==============================] - 18s 56ms/step - loss: 0.1301 - accuracy: 0.9591
Epoch 4/5
312/313 [============================>.] - ETA: 0s - loss: 0.0782 - accuracy: 0.9777

In [ ]:
y_pred = model.predict_classes(x_test, batch_size = 128)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

Correct Prediction: 8669
Wrong Prediction: 1331
Accuracy: 86.69
